# One-dimensional (1D) Fourier transformation

In this notebook, we are going to transform time-domain data into 1D or 2D spectra using SpectroChemPy
processing tools

In [ ]:
import spectrochempy as scp

## FFT of 1D NMR spectra

First we open read some time domain data. Here is a NMD free induction decay (FID):

In [ ]:
path = scp.preferences.datadir / "nmrdata" / "bruker" / "tests" / "nmr" / "topspin_1d"
fid = scp.read_topspin(path)
fid

The type of the data is complex:

In [ ]:
fid.dtype

We can represent both real and imaginary parts on the same plot using the `show_complex` parameter.

In [ ]:
prefs = scp.preferences
prefs.figure.figsize = (6, 3)
fid.plot(show_complex=True, xlim=(0, 15000))
print("td = ", fid.size)

Now we perform a Fast Fourier Transform (FFT):

In [ ]:
spec = scp.fft(fid)
spec.plot(xlim=(100, -100))
print("si = ", spec.size)
spec

**Alternative notation**

In [ ]:
k = 1024
spec = fid.fft(size=32 * k)
spec.plot(xlim=(100, -100))
print("si = ", spec.size)

In [ ]:
newfid = spec.ifft()
# x coordinate is in second (base units) so lets transform it
newfid.plot(show_complex=True, xlim=(0, 15000))

Let's compare fid and newfid. There differs as a rephasing has been automatically applied after the first FFT (with
the parameters found in the original fid metadata: PHC0 and PHC1).

First point in the time domain of the real part is at the maximum.

In [ ]:
newfid.real.plot(c="r", label="fft + ifft")
ax = fid.real.plot(clear=False, xlim=(0, 5000), ls="--", label="original real part")
_ = ax.legend()

First point in the time domain of the imaginary part is at the minimum.

In [ ]:
fid.imag.plot(ls="--", label="original imaginary part")
ax = newfid.imag.plot(clear=False, xlim=(0, 5000), c="r", label="fft + ifft")
_ = ax.legend(loc="lower right")

## Preprocessing

### Line broadening
Often before applying FFT, some exponential multiplication `em`or other broadening filters such as `gm` or `sp`
are applied.
See the dedicated [apodization tutorial](apodization.rst).

In [ ]:
fid2 = fid.em(lb="50. Hz")
spec2 = fid2.fft()
spec2.plot()
spec.plot(
    clear=False, xlim=(10, -5), c="r"
)  # superpose the non broadened spectrum in red and show expansion.

### Zero-filling

In [ ]:
print("td = ", fid.size)

In [ ]:
td = 64 * 1024  # size: 64 K
fid3 = fid.zf_size(size=td)
print("new td = ", fid3.x.size)

In [ ]:
spec3 = fid3.fft()
spec3.plot(xlim=(100, -100))
print("si = ", spec3.size)

### Time domain baseline correction
See the dedicated [Time domain baseline correction tutorial](td_baseline.rst).

### Magnitude calculation

In [ ]:
ms = spec.mc()
ms.plot(xlim=(10, -10))
spec.plot(clear=False, xlim=(10, -10), c="r")

### Power spectrum

In [ ]:
mp = spec.ps()
(mp / mp.max()).plot()
(spec / spec.max()).plot(
    clear=False, xlim=(10, -10), c="r"
)  # Here we have normalized the spectra at their max value.

# Real Fourier transform

In some case, it might be interesting to perform real Fourier transform . For instance, as a demonstration,
we will independently transform real and imaginary part of the previous fid, and recombine them to obtain the same
result as when performing complex fourier transform on the complex dataset.

In [ ]:
lim = (-20, 20)
spec3.plot(xlim=lim)
spec3.imag.plot(xlim=lim)

In [ ]:
Re = fid3.real.astype("complex64")
fR = Re.fft()
fR.plot(xlim=lim, show_complex=True)
Im = fid3.imag.astype("complex64")
fI = Im.fft()
fI.plot(xlim=lim, show_complex=True)

Recombination:

In [ ]:
(fR - fI.imag).plot(xlim=lim)
(fR.imag + fI).plot(xlim=lim)